# Open NWM 1km dataset as DFReferenceFileSystem 

Open dataset as a fsspec `DFReferenceFileSystem` filesystem by reading references from a collection of Parquet files: one file containing global metadata and coordinate variable references, and one file for each of the data variables.  

The big wins here are lazy-loading of the references for each variable, and the more efficient construction of the virtual fsspec filesystem from the Parquet files (JSON is slow to decode).

In [2]:
import fsspec
from fsspec.implementations.reference import DFReferenceFileSystem
import xarray as xr

In [3]:
fs = fsspec.filesystem('s3', anon=True, 
                        client_kwargs={'endpoint_url':'https://ncsa.osn.xsede.org'})

In [4]:
s3_lazy_refs = 's3://esip/noaa/nwm/lazy_refs'

In [8]:
print(f'Number of reference files: {len(fs.ls(s3_lazy_refs))}')
print(f'Total size of references: {fs.du(s3_lazy_refs)/1e9} GB')

Number of Parquet files: 21
Total size of Parquet references: 0.492091486 GB


In [9]:
r_opts = {'anon': True}
t_opts = {'anon': True, 'client_kwargs':{'endpoint_url':'https://ncsa.osn.xsede.org'}}

In [10]:
%%time
fs2 = DFReferenceFileSystem(s3_lazy_refs, lazy=True, target_options=t_opts,
                        remote_protocol='s3', remote_options=r_opts)
m = fs2.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", chunks={}, backend_kwargs=dict(consolidated=False))

CPU times: user 5.03 s, sys: 404 ms, total: 5.43 s
Wall time: 12 s


In [11]:
ds

<xarray.Dataset>
Dimensions:   (time: 116631, y: 3840, x: 4608, vis_nir: 2, soil_layers_stag: 4)
Coordinates:
  * time      (time) datetime64[ns] 1979-02-01T03:00:00 ... 2020-12-31T21:00:00
  * x         (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y         (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Dimensions without coordinates: vis_nir, soil_layers_stag
Data variables: (12/21)
    ACCET     (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    ACSNOM    (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    ALBEDO    (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    ALBSND    (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 960, 1, 1152), meta=np.ndarray>
    ALBSNI    (time, y, vis_nir, x) float64 dask.array<chunksize=(1, 960, 1, 1152), meta=np.ndarray>
    COSZ      (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    ...        ...
    SNOWH     (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    SOIL_M    (time, y, soil_layers_stag, x) float64 dask.array<chunksize=(1, 768, 1, 922), meta=np.ndarray>
    SOIL_W    (time, y, soil_layers_stag, x) float64 dask.array<chunksize=(1, 768, 1, 922), meta=np.ndarray>
    TRAD      (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    UGDRNOFF  (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    crs       object ...
Attributes:
    Conventions:                CF-1.6
    GDAL_DataType:              Generic
    TITLE:                      OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:               v5.2.0-beta2
    model_configuration:        retrospective
    model_initialization_time:  1979-02-01_00:00:00
    model_output_type:          land
    model_output_valid_time:    1979-02-01_03:00:00
    model_total_valid_times:    472
    proj4:                      +proj=lcc +units=m +a=6370000.0 +b=6370000.0 ...

Examine a specific variable:

In [12]:
ds.TRAD

<xarray.DataArray 'TRAD' (time: 116631, y: 3840, x: 4608)>
dask.array<open_dataset-6e252a6bf6e4556824b3940793011076TRAD, shape=(116631, 3840, 4608), dtype=float64, chunksize=(1, 768, 922), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1979-02-01T03:00:00 ... 2020-12-31T21:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    grid_mapping:    crs
    long_name:       Surface radiative temperature
    units:           K
    valid_range:     [0, 4000]

How big would the uncompressed size of the whole dataset be?

In [13]:
ds.nbytes/1e12  #TB

462.28064798432

Load some data at a specific time step.  The first time a variable is accessed it will take longer as the references need to be loaded.

In [14]:
%%time 
da = ds.TRAD.sel(time='1990-01-01 00:00').load()

CPU times: user 7.12 s, sys: 1.26 s, total: 8.39 s
Wall time: 13.6 s


Loading data for another time step is much faster as the references are already loaded:

In [15]:
%%time
da = ds.TRAD.sel(time='2015-01-01 00:00').load()

CPU times: user 4.04 s, sys: 531 ms, total: 4.57 s
Wall time: 6.31 s


Compute the mean over the domain:

In [16]:
da.mean().data

array(266.92635398)

In [ ]:
da.plot()